
---

# 🧭 Discriminated Unions

### 🎯 Intent

Handle **different shapes of data** in one field, where a **discriminator key** decides which model to use.

---

### 🧩 Core Components

1. **🏷️ Discriminator Field**

   * A special key (like `kind`) in every variant.
   * Example: `"card"` vs `"upi"`.

2. **🔀 Variants as Models**

   * Each option is its own `BaseModel`.
   * Combined with `Union[...]` or `|`.

3. **🧩 `Field(discriminator="...")`**

   * Tells Pydantic which field is the switch.
   * Example: `payment: Union[Card, Upi] = Field(..., discriminator="kind")`.

4. **✅ How It Works**

   * Looks at `kind` → picks the right model.
   * Gives **clear errors** if wrong type is passed.

5. **📤 Serialization & Schema**

   * `model_dump()` → returns the selected variant.
   * `model_json_schema()` → generates `oneOf` with discriminator mapping.

6. **🧱 Best Practices**

   * Use **stable literal values** (`Literal["card","upi"]`).
   * Keep `kind` **required**.
   * Choose **clear labels** for long-term use.

---


In [1]:
from typing import Union, Literal
from pydantic import BaseModel, Field, ValidationError

# ✅ Variants
class CardPayment(BaseModel):
    kind: Literal["card"]
    card_number: str
    amount: float

class UpiPayment(BaseModel):
    kind: Literal["upi"]
    upi_id: str
    amount: float

# ✅ Union with discriminator
class PaymentRequest(BaseModel):
    payment: Union[CardPayment, UpiPayment] = Field(..., discriminator="kind")

# ⚡ Usage
try:
    # Correct: card variant
    card = PaymentRequest.model_validate({
        "payment": {"kind": "card", "card_number": "4111111111111111", "amount": 100.0}
    })
    print("Card OK:", card.model_dump())

    # Correct: upi variant
    upi = PaymentRequest.model_validate({
        "payment": {"kind": "upi", "upi_id": "mukesh@upi", "amount": 250.0}
    })
    print("UPI OK:", upi.model_dump())

    # ❌ Wrong discriminator
    bad = PaymentRequest.model_validate({
        "payment": {"kind": "paypal", "email": "test@example.com", "amount": 50}
    })
except ValidationError as e:
    print("Validation failed:", e.errors())


Card OK: {'payment': {'kind': 'card', 'card_number': '4111111111111111', 'amount': 100.0}}
UPI OK: {'payment': {'kind': 'upi', 'upi_id': 'mukesh@upi', 'amount': 250.0}}
Validation failed: [{'type': 'union_tag_invalid', 'loc': ('payment',), 'msg': "Input tag 'paypal' found using 'kind' does not match any of the expected tags: 'card', 'upi'", 'input': {'kind': 'paypal', 'email': 'test@example.com', 'amount': 50}, 'ctx': {'discriminator': "'kind'", 'tag': 'paypal', 'expected_tags': "'card', 'upi'"}, 'url': 'https://errors.pydantic.dev/2.11/v/union_tag_invalid'}]
